In [4]:
from sklearn import svm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [5]:
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [6]:
XY = pd.read_pickle('MLbodyFace').dropna()

In [7]:
XY.columns

Index([u'seq', u'id', u'traSeq', u'trackId', u'trackX', u'trackY', u'trackW',
       u'trackH', u'height', u'distance', u'rgbSeq', u'blur', u'embedding',
       u'GroundTrue'],
      dtype='object')

In [8]:
XY.drop(['seq', 'id', 'traSeq', 'trackId', 'rgbSeq'], axis=1, inplace=True)

In [9]:
X = XY.reset_index().drop('GroundTrue', 1)

In [10]:
X

,index,trackX,trackY,trackW,trackH,height,distance,blur,embedding
0,0,326.4,124.0,387.6,458.0,1.571027,2.248792,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."
1,1,802.4,150.0,442.0,520.0,1.519628,1.882822,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,..."
2,2,1135.6,108.0,309.4,364.0,1.475796,2.760290,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ..."
3,4,323.0,124.0,387.6,456.0,1.569231,2.253340,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."
4,5,805.8,152.0,438.6,518.0,1.519507,1.894786,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,..."
5,6,1128.8,104.0,312.8,370.0,1.486020,2.723061,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ..."
6,12,306.0,124.0,387.6,458.0,1.567878,2.250316,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."
7,13,809.2,152.0,438.6,516.0,1.524655,1.898137,790.153174,"[0.0554316, 0.0860607, -0.00389893, 0.0320879,..."
8,14,1125.4,104.0,316.2,374.0,1.490240,2.710115,600.989129,"[-0.0656685, 0.047579, -0.100778, 0.00661445, ..."
9,16,306.0,124.0,387.6,458.0,1.565820,2.250340,367.316834,"[-0.110296, 0.0504573, 0.00468473, -0.0105098,..."


In [11]:
em = [[] for i in range(len(X.iloc[0].embedding))]

for index, r in X.iterrows():
    for column in range (len(X.iloc[0].embedding)):
         em[column].append(r.embedding[column])

In [12]:
data = np.asarray(em).T

In [13]:
for column in range (len(X.iloc[0].embedding)):
    X[column] = em[column]

In [14]:
X.drop('embedding', 1, inplace=True)

In [15]:
Y = XY.GroundTrue

In [32]:
bestC = 100
bestMean = 0

for i in range(83, 85, 1):
    print '**************************'
    oneVsAll = OneVsRestClassifier(LinearSVC(C=i))
    cross = cross_val_score(oneVsAll, X, Y, cv=10, verbose=10)
    mean = cross.mean()
    if bestMean < mean:
        bestC = i
        bestMean = mean

**************************
[CV]  ................................................................
[CV] ................................. , score=0.500000, total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV] ................................. , score=0.230769, total=   1.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.1s remaining:    0.0s


[CV] ................................. , score=0.296703, total=   1.1s
[CV]  ................................................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    8.8s remaining:    0.0s


[CV] ................................. , score=0.288889, total=   1.0s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    9.7s remaining:    0.0s


[CV] ................................. , score=0.460674, total=   1.0s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   10.8s finished


In [33]:
bestC

84

In [34]:
bestMean

0.41279513245805377

In [17]:
cross = cross_val_score(oneVsAll, X, Y, cv=10, verbose=100)

[CV]  ................................................................
[CV] ................................. , score=0.293478, total=   1.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.285714, total=   1.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.4s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.494505, total=   1.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.5s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , score=0.422222, total=   1.1s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.5s remaining:    0.0s
[CV]  ................................................................
[CV] ................................. , 

In [18]:
cross.mean()

0.38915507252341647

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)

In [ ]:
#polynomial_svm.fit(X_train, y_train)

In [ ]:
#polynomial_svm.score(X_test, y_test)